NAME: YASH PATNI

ROLL NO.: A4-71

PRACTICAL NO. 4

AIM:

(A) Write a program to validate a string using the parsing table generated in
practical 3. Consider valid & invalid string & print the stack & buffer input tracing.

Input: LL (1) parsing table

Implementation: String parsing rules using stack

Output: Each step-in string parsing and whether the input string is valid or invalid.


In [10]:
from tabulate import tabulate

terminals = ['a', 'b', 'c', 'p']
non_terminals = ['S', 'A', 'B', 'C']
starting_symbol = 'S'
productions_dict = {
    'S': ['A', 'BC'],
    'A': ['a', 'b'],
    'B': ['p', '#'],
    'C': ['c']
}

LL1_table = {}
conflicts = False

def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]
        for alternative in alternatives:
            first_2 = first(alternative)
            first_ |= first_2
    elif string in terminals:
        first_ = {string}
    elif string == '' or string == '#':
        first_ = {'#'}
    else:
        first_2 = first(string[0])
        if '#' in first_2:
            i = 1
            while '#' in first_2:
                first_ |= (first_2 - {'#'})
                if string[i:] in terminals:
                    first_ |= {string[i:]}
                    break
                elif string[i:] == '':
                    first_ |= {'#'}
                    break
                first_2 = first(string[i:])
                first_ |= (first_2 - {'#'})
                i += 1
        else:
            first_ |= first_2
    return first_

def follow(nT):
    follow_ = set()
    prods = productions_dict.items()
    if nT == starting_symbol:
        follow_ |= {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for char in alt:
                if char == nT:
                    following_str = alt[alt.index(char) + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ |= follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if '#' in follow_2:
                            follow_ |= (follow_2 - {'#'})
                            follow_ |= follow(nt)
                        else:
                            follow_ |= follow_2
    return follow_

FIRST = {non_terminal: set() for non_terminal in non_terminals}
FOLLOW = {non_terminal: set() for non_terminal in non_terminals}

for non_terminal in non_terminals:
    FIRST[non_terminal] |= first(non_terminal)

FOLLOW[starting_symbol] |= {'$'}
for non_terminal in non_terminals:
    FOLLOW[non_terminal] |= follow(non_terminal)

for non_terminal, alternatives in productions_dict.items():
    for alternative in alternatives:
        first_set_alt = first(alternative)
        for terminal in first_set_alt - {'#'}:
            if (non_terminal, terminal) not in LL1_table:
                LL1_table[(non_terminal, terminal)] = alternative
            else:
                print(f"Conflict at ({non_terminal}, {terminal})")
                conflicts = True

        if '#' in first_set_alt or '' in first_set_alt:  # If epsilon is in FIRST(α)
            for terminal in FOLLOW[non_terminal]:
                if (non_terminal, terminal) not in LL1_table:
                    LL1_table[(non_terminal, terminal)] = alternative
                else:
                    print(f"Conflict at ({non_terminal}, {terminal})")
                    conflicts = True

if not conflicts:
    print("No conflicts found.")

# Parsing function
def parse_input(input_str):
    stack = [starting_symbol]
    buffer = list(input_str)
    stack_trace = []
    buffer_trace = [buffer[:]]
    action_trace = []
    accepted = False

    while stack:
        top_of_stack = stack[-1]
        if top_of_stack in terminals:
            if top_of_stack == buffer[0]:
                stack.pop()
                buffer.pop(0)
                action_trace.append(f"Matched {top_of_stack}")
            else:
                break
        elif top_of_stack in non_terminals:
            key = (top_of_stack, buffer[0])
            if key in LL1_table:
                production = LL1_table[key]
                stack.pop()
                if production != '#':
                    action_trace.append(f"Applied {top_of_stack} -> {production}")
                    stack.extend(reversed(production))
            else:
                break
        else:
            break

        stack_trace.append(''.join(stack))
        buffer_trace.append(buffer[:])

        if not stack and not buffer:
            accepted = True

    return accepted, stack_trace, buffer_trace, action_trace

# Validate strings
def validate_string(input_str):
    print("Parsing input string:", input_str)
    accepted, stack_trace, buffer_trace, action_trace = parse_input(input_str)

    if accepted:
        print("String Accepted.")
    else:
        print("String Rejected.")

    trace_data = list(zip(stack_trace, buffer_trace, action_trace))
    print(tabulate(trace_data, headers=['Stack Trace', 'Buffer Trace', 'Action Trace'], tablefmt='grid'))

# Test with some strings
validate_string("pc")
validate_string("abbc")


No conflicts found.
Parsing input string: pc
String Accepted.
+---------------+----------------+-----------------+
| Stack Trace   | Buffer Trace   | Action Trace    |
+===============+================+=================+
| CB            | ['p', 'c']     | Applied S -> BC |
+---------------+----------------+-----------------+
| Cp            | ['p', 'c']     | Applied B -> p  |
+---------------+----------------+-----------------+
| C             | ['p', 'c']     | Matched p       |
+---------------+----------------+-----------------+
| c             | ['c']          | Applied C -> c  |
+---------------+----------------+-----------------+
|               | ['c']          | Matched c       |
+---------------+----------------+-----------------+
Parsing input string: abbc
String Rejected.
+---------------+----------------------+----------------+
| Stack Trace   | Buffer Trace         | Action Trace   |
+===============+======================+================+
| A             | ['a', 'b', 'b